In [21]:
%matplotlib notebook

import time
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

start_time = time.time()

class Result:
    def __init__(self, read_name, kindom, genus, speices, align_len, read_cover, ident):
        self.read_name = read_name
        self.kindom = kindom
        self.genus = genus
        self.speices = speices
        self.align_len = align_len
        self.read_cover = read_cover
        self.ident = ident
    def __str__(self):
        return self.read_name + " " + self.kindom + " " + self.genus + " " + self.speices + " " +\
                self.align_len + " " + self.read_cover + " " + self.ident
    
def file2Result(file_name):
    results=[]
    with open(file_name) as f:
        for line in f:
            tmp = line.split()
            results.append(Result(tmp[0], tmp[1], tmp[2], tmp[3], float(tmp[4]), float(tmp[5]), float(tmp[6])))
    return results

def name_count_dict(align_result, fhit_kindom_hash, fhit_genus_hash, threshold=(0,0,0), merge_lb = 10000, ):
    add_Other_category = False
    al_threshold, rc_threshold, i_threshold, num_threshold = threshold
    
    for obj in align_result:
        if obj.align_len >= al_threshold and obj.read_cover >= rc_threshold and obj.ident >= i_threshold:
            if obj.kindom in fhit_kindom_hash:
                fhit_kindom_hash[obj.kindom] += 1
            else:
                fhit_kindom_hash[obj.kindom] = 1

            if (obj.kindom + " " + obj.genus) in fhit_genus_hash:
                fhit_genus_hash[obj.kindom + " " + obj.genus] += 1
                if (not add_Other_category) and fhit_genus_hash[obj.kindom + " " + obj.genus] >= merge_lb:
                    add_Other_category = True
            else:
                fhit_genus_hash[obj.kindom + " " + obj.genus] = 1
    #add Other category : if exists a "genus" number larger than threshold, add "Other" category 
    #all "genus" number less than threshold merge to "Other" 
    if add_Other_category:
        fhit_genus_hash["Other Other"] = 0
        for name in list(fhit_genus_hash.keys()):
            if fhit_genus_hash[name] < merge_lb and name != "Other Other":
                fhit_genus_hash["Other Other"] += fhit_genus_hash[name]
                del fhit_genus_hash[name]
    okfhit = OrderedDict(sorted(fhit_kindom_hash.items(), key=lambda t: t[1], reverse=True ))
    ogfhit = OrderedDict(sorted(fhit_genus_hash.items(), key=lambda t: t[1], reverse=True ))
    genus_count = 0
    if add_Other_category:
        if "Other Other" not in ogfhit: ogfhit["Other Other"] = 0
        for k,v in ogfhit.items():
            if k != "Other Other": genus_count += 1
            if genus_count > num_threshold and k != "Other Other":
                ogfhit["Other Other"] += v
                del ogfhit[k]     
    return (add_Other_category, okfhit, ogfhit)

def print_table(ordered_kindom_fhit, ordered_genus_fhit, add_Other_category):
    print("First Hit Kindom Table\n")
    print('   {0:<15}  {1:<10}  {2:<10}'.format("Kindom", "num", "ratio"))
    for k,v in ordered_kindom_fhit.items():
        print('   {0:<15}  {1:<10}  {2:<5.2f}{3}'.format(k, v, round(v/total_align_num, 4)*100, "%"))
    print("\n")
    print("First Hit Genus Table\n")
    print('   {0:<20} {1:<20}  {2:<10}  {3:<10}'.format("Kindom", "Genus", "num", "ratio"))
    for k,v in ordered_genus_fhit.items():
        if k != "Other Other":
            print('   {0:<20} {1:<20}  {2:<10}  {3:<5.2f}{4}'.format((k.split())[0], (k.split())[1], v, round(v/total_align_num, 4)*100, "%"))
    if add_Other_category:        
        print('   {0:<20} {1:<20}  {2:<10}  {3:<5.2f}{4}'.format\
              ("Other", "Other", ordered_genus_fhit["Other Other"], round(ordered_genus_fhit["Other Other"]/total_align_num, 4)*100, "%"))
    print("\n")
    print("\n")


def scatter_plot(align_result):
    small_align_len=[]
    small_read_cover=[]
    small_ident=[]

    #pick first 10000 entry to draw scatter plot
    for i,obj in enumerate(align_result):
        if 8000 < i < 10000 and 50000 < i < 52000 or 70000 < i < 72000 or 100000 < i < 102000 or 200000 < i < 202000:
            small_align_len.append(int(obj.align_len))
            small_ident.append(float(obj.ident))
            small_read_cover.append(float(obj.read_cover))

    scatter_fig1, alignlen_rcover = plt.subplots()
    x = small_align_len
    y = small_read_cover
    alignlen_rcover.scatter(x, y, s=5)
    alignlen_rcover.set_xlim(left=0,right=10000)
    alignlen_rcover.set_ylim((0,1))
    alignlen_rcover.xaxis.set_ticks(np.arange(0, 10000, 200))
    alignlen_rcover.yaxis.set_ticks(np.arange(0, 1.1, 0.1))
    alignlen_rcover.set_xlabel("alignment len")
    alignlen_rcover.set_ylabel("read cover")
    alignlen_rcover.set_title("alignment len & read cover")

    scatter_fig2, alignlen_ident = plt.subplots()
    x = small_align_len
    y = small_ident
    alignlen_ident.scatter(x, y, s=5)
    alignlen_ident.set_xlim(left=0, right=10000)
    alignlen_ident.set_ylim((0,1))
#     alignlen_ident.xaxis.set_ticks(np.arange(0, 3600, 100))
    alignlen_ident.yaxis.set_ticks(np.arange(0, 1.1, 0.1))
    alignlen_ident.set_xlabel("alignment len")
    alignlen_ident.set_ylabel("identity")
    alignlen_ident.set_title("alignment len & identity")

    scatter_fig3, rcover_ident = plt.subplots()
    x = small_read_cover
    y = small_ident
    rcover_ident.scatter(x, y, s=5)
    rcover_ident.set_xlim((0,1))
    rcover_ident.set_ylim((0,1))
    rcover_ident.xaxis.set_ticks(np.arange(0, 1.1, 0.1))
    rcover_ident.yaxis.set_ticks(np.arange(0, 1.1, 0.1))
    rcover_ident.set_xlabel("read cover")
    rcover_ident.set_ylabel("identity")
    rcover_ident.set_title("read cover & identity")
    
def pie_plot(ordered_dict, split_pos, plot_name):    
    fhit_fig, ax = plt.subplots(figsize=(8,4))    
    fhit_label = []
    fhit_size = []
    for k in ordered_dict:
        fhit_label.append((k.split())[split_pos])
        fhit_size.append(ordered_dict[k])
    wedges, texts, autotext = ax.pie(fhit_size, autopct='%1.1f%%', textprops={'fontsize': 9,'color': 'w'},
                                     counterclock=False, startangle=90, pctdistance=0.7)
    for i,text in enumerate(autotext):
        fhit_label[i] = '{0}  {1}'.format(fhit_label[i],autotext[i].get_text()) 

    legend = ax.legend(wedges, fhit_label, loc="right", bbox_to_anchor=(1.1,0.5))
    vp = legend._legend_box._children[-1]._children[0]
    for c in vp._children:
        c._children.reverse() 
    vp.align="right" 
    ax.axis('equal')
    ax.set_title(plot_name)

# parse file to Result class    
align_result = file2Result("/big6_disk/fangren106/mock_community/zymo_even/alignment/R10/uniq_map.analysis")

######### threshold #########

align_len_threshold = 0
read_cover_threshold = 0
ident_threshold = 0

##############################

##### category threshold #####

merge_lower_bound = 0
num_threshold = 10

##############################

thresholds = (align_len_threshold, read_cover_threshold, ident_threshold, num_threshold)
total_align_num = len(align_result)
print("Total alignment number: ", total_align_num, "\n")

# put every result into dict{name : result number}
# merge result less than merge_lower_bound into "Other" category
fhit_kindom_hash={}
fhit_genus_hash={}

(add_Other_category, order_kfhit, order_gfhit) = name_count_dict(align_result, fhit_kindom_hash, fhit_genus_hash,
                                               thresholds, merge_lower_bound)


# print data table
print_table(order_kfhit, order_gfhit, add_Other_category)

######## plot ####### 

# pie_plot(order_kfhit, 0, "first hit Kindom")
# pie_plot(order_gfhit, 1, "first hit Genus")
# scatter_plot(align_result)

####################

print("--- %s seconds ---" % (time.time() - start_time))


Total alignment number:  2767609 

First Hit Kindom Table

   Kindom           num         ratio     
   Bacteria         2767609     100.00%


First Hit Genus Table

   Kindom               Genus                 num         ratio     
   Bacteria             Bacillus              687661      24.85%
   Bacteria             Listeria              512397      18.51%
   Bacteria             Enterococcus          425536      15.38%
   Bacteria             Staphylococcus        412772      14.91%
   Bacteria             Lactobacillus         273201      9.87 %
   Bacteria             Escherichia           154636      5.59 %
   Bacteria             Salmonella            133921      4.84 %
   Bacteria             Saccharomyces         68833       2.49 %
   Bacteria             Cryptococcus          54887       1.98 %
   Bacteria             Pseudomonas           43765       1.58 %
   Other                Other                 0           0.00 %




--- 28.79671621322632 seconds ---
